# 🚀 GraphQL to MCP Starterkit

**Do you want to use your GraphQL APIs with agents? Let's give it a shot.**

## 📋 Goals

### The Basics
1. **GraphQL ↔ AI Concepts**: How schemas become tool contracts
2. **Safety First**: Queries vs mutations in AI context  
3. **Hands-On Examples**: Real GraphQL APIs → MCP

## Connect your GQL APIs to real AI apps
4. **AI Client Integration**: Claude & Cursor setup
5. **Schema Design**: Writing AI-friendly documentation

## Beyond the homelab setup
6. **Production Ready**: Authentication, governance, monitoring
7. **Toolprint Integration**: Smart tool orchestration

## 🎯 Prerequisites

- Basic GraphQL knowledge (queries, mutations, schemas)
- Command line comfort
- Node.js installed
- An existing GraphQL API (or we'll use examples)


---


## 🧠 Chapter 1: GraphQL Meets AI - The Conceptual Bridge

### Why GraphQL + AI = Perfect Match

GraphQL APIs are uniquely suited for AI tool integration because they are:

- **🔍 Self-Documenting**: Schemas contain type information and descriptions
- **🎯 Strongly Typed**: Clear parameter validation and return types  
- **🔎 Introspectable**: APIs can describe themselves programmatically
- **⚡ Efficient**: Single endpoint, precise field selection

### The Challenge: Raw GraphQL ≠ AI-Ready Tools

However, GraphQL APIs aren't immediately AI-accessible:

```graphql
# This GraphQL query...
query GetUser($id: ID!) {
  user(id: $id) {
    id
    name
    posts {
      title
      author {  # 🚨 Circular reference!
        name
      }
    }
  }
}
```

**AI agents struggle with:**
- Writing complex field selections
- Handling circular references  
- Understanding parameter requirements
- Knowing which tools exist


### The MCP Solution: Universal Tool Protocol

**Model Context Protocol (MCP)** bridges this gap by providing a standardized way for AI tools to communicate with data sources and APIs.

The solution requires something that can:
1. **Introspect the GraphQL schema** to understand available operations
2. **Generate individual tools** for each query and mutation
3. **Handle field selection automatically** to avoid GraphQL validation errors
4. **Provide proper parameter validation** before execution


<img alt="Basic architecture of a MCP <> GraphQL Proxy" src="./assets/mcp-graphql-proxy.png" width="100%" height="100%">

### 🍱 Some Food for Thought

Before we dive into implementation, there are important caveats to consider:

#### 1. **Mutation Safety**: Should mutations be exposed as tools?
```graphql
# This becomes a tool that could delete data!
type Mutation {
  deleteUser(id: ID!): Boolean
  chargeCard(amount: Float!): PaymentResult  
}
```
**Question**: Should AI agents have unrestricted access to write operations?

#### 2. **Data Shape**: How much data should we return?
```graphql
# Minimal - fast, fits in context
{ id, name }

# Expansive - comprehensive, but large
{ id, name, email, posts { title, content, comments { author, text } } }
```
**Consideration**: Context window limits vs completeness

#### 3. **Query Composition**: Some queries only make sense together
```graphql
# Step 1: Get user ID
user(username: "john") { id }

# Step 2: Get user's posts (needs ID from step 1)  
posts(authorId: "user-123") { title }
```
#### 4. **Challenge**: Individual tools may not provide complete workflows

## 🔬 Chapter 3: DEMO - make a real GraphQL service an MCP

Option 1 - **connect to toolprint graphql api without authentication**
_Note that this may take a few seconds to load up because it's running on Strapi's free tier_

This will be your environment variable for the next step:
```
export GRAPHQL_ENDPOINT=https://patient-song-383e3762d2.strapiapp.com/graphql
```

In [ ]:
GRAPHQL_ENDPOINT=https://patient-song-383e3762d2.strapiapp.com/graphql \
npx -y @toolprint/mcp-graphql-forge --transport http --port 3001

Option 2 - **connect to the github graphql API**
1. Grab your existing Github Personal Access Token (! Ensure that it only has READ perms for safety) OR [follow this guide](https://docs.github.com/en/rest/authentication/authenticating-to-the-rest-api?apiVersion=2022-11-28) to create a new one. 

These will be your environment variables for the next step:
```
export GRAPHQL_ENDPOINT=https://api.github.com/graphql
export GRAPHQL_AUTH_HEADER=Bearer ${GH_TOKEN}
```

In [ ]:
GRAPHQL_ENDPOINT=https://api.github.com/graphql \
GRAPHQL_AUTH_HEADER=Bearer ${GH_TOKEN} \
npx -y @toolprint/mcp-graphql-forge --transport http --port 3001

Now we'll inspect our tools using [mcp inspector](https://github.com/modelcontextprotocol/inspector)
When we launch the inspector, grab the URL outputted in the terminal and open it up. Then, connect to our launched MCP <> GraphQL Proxy on the connection menu like so:

<img alt="Basic architecture of a MCP <> GraphQL Proxy" src="./assets/connect-via-inspector.png" width="30%" height="30%">

**Note that our mcp <> graphql proxy is using the Streamable HTTP transport via the --transport http flag**

## Exploring Our Tools in MCP Inspector



In [ ]:
# 🎬 LIVE DEMO: Watch the GraphQL → MCP transformation happen

# Let's use our real Strapi CMS GraphQL API to see all the concepts in action
GRAPHQL_ENDPOINT="https://patient-song-383e3762d2.strapiapp.com/graphql" \
npx @toolprint/mcp-graphql-forge --transport http --port 3001

# Watch for these key outputs:
# 1. "Introspecting GraphQL schema..." - Schema analysis
# 2. "🗂️ Building field selection cache..." - Solving the field selection problem  
# 3. "📊 Generated field selections for X types" - Circular reference handling
# 4. "Generated Y tools from GraphQL schema" - Individual tools created
# 5. Tool breakdown: queries vs mutations

# Strapi typically generates rich schemas with:
# - Content types (Articles, Users, Categories, etc.)
# - Nested relationships between entities
# - Comprehensive field selections
# - Both queries and mutations for CRUD operations


In [ ]:
## 🤖 Chapter 4: Connecting to AI Clients (Claude/Cursor)

Now that we've tested our GraphQL-to-MCP bridge, let's connect it to actual AI clients where the real magic happens.

### The Transport Switch: HTTP → Stdio

**Important**: AI clients like Claude and Cursor use **stdio transport**, not HTTP.

- **HTTP Transport** = Great for development, testing with Inspector
- **Stdio Transport** = Required for AI client integration

### Step 1: Configure for Claude Desktop

Create or edit your `claude_desktop_config.json` file:

**Location**:
- **macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`
- **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`


In [ ]:
{
  "mcpServers": {
    "strapi-cms": {
      "command": "npx",
      "args": [
        "-y",
        "@toolprint/mcp-graphql-forge"
      ],
      "env": {
        "GRAPHQL_ENDPOINT": "https://patient-song-383e3762d2.strapiapp.com/graphql"
      }
    }
  }
}


In [ ]:
### Step 2: Configure for Cursor

Create or edit your `mcp.json` file in your project root:


In [ ]:
{
  "mcpServers": {
    "strapi-cms": {
      "command": "npx",
      "args": [
        "-y", 
        "@toolprint/mcp-graphql-forge"
      ],
      "env": {
        "GRAPHQL_ENDPOINT": "https://patient-song-383e3762d2.strapiapp.com/graphql"
      }
    }
  }
}


In [ ]:
### Step 3: Test the Integration

#### For Claude Desktop:
1. **Save the config file** and restart Claude Desktop
2. **Look for the 🔌 icon** in the Claude interface
3. **Start a conversation**: "What articles are available in the CMS?"
4. **Watch Claude use the tools** automatically!

#### For Cursor:
1. **Save the mcp.json file** in your project root
2. **Restart Cursor** and open the project
3. **Ask about your CMS**: "Show me the latest articles from the CMS"
4. **See Cursor query your GraphQL API** through the generated tools

### 🎯 Test Prompts to Try

#### Safe Exploration (Queries Only)
- "What content types are available in the CMS?"
- "Show me the 5 most recent articles"
- "List all categories and their article counts"
- "Find articles containing 'tutorial' in the title"

#### Advanced Analysis  
- "Analyze the content strategy - which categories have the most articles?"
- "Create a summary of all article titles and their publication dates"
- "What are the top authors by article count?"

### 🛡️ Safety Note

Remember: The tools generated include both **queries** (safe) and **mutations** (potentially destructive). For production use, consider:
- Starting with query-only mode
- Adding authentication requirements
- Implementing audit logging for mutations


In [ ]:
# 🚀 From GraphQL APIs to AI Tools: A Complete MCP Guide

**Transform your existing GraphQL APIs into AI-accessible tools with Model Context Protocol (MCP)**

---


## 📋 What You'll Learn

1. **GraphQL ↔ AI Concepts**: How schemas become tool contracts
2. **Safety First**: Queries vs mutations in AI context  
3. **Hands-On Examples**: Real GraphQL APIs → AI tools
4. **Schema Design**: Writing AI-friendly documentation
5. **Production Ready**: Authentication, governance, monitoring
6. **Toolprint Integration**: Smart tool orchestration

---

## 🎯 Prerequisites

- Basic GraphQL knowledge (queries, mutations, schemas)
- Command line comfort
- Node.js installed
- An existing GraphQL API (or we'll use examples)


In [ ]:
## 🧠 Chapter 1: GraphQL Meets AI - The Conceptual Bridge

### Why GraphQL + AI = Perfect Match

### The Challenge: Raw GraphQL ≠ AI-Ready Tools

However, GraphQL APIs aren't immediately AI-accessible:

```graphql
# This GraphQL query...
query GetUser($id: ID!) {
  user(id: $id) {
    id
    name
    posts {
      title
      author {  # 🚨 Circular reference!
        name
      }
    }
  }
}
```

**NOT MCP FRIENDLY**

**AI agents struggle with:**
- Writing complex field selections
- Handling circular references  
- Understanding parameter requirements
- Knowing which tools exist


In [ ]:
### The MCP Solution: Universal Tool Protocol

**Model Context Protocol (MCP)** bridges this gap by providing a standardized way for AI tools to communicate with data sources and APIs.

The solution requires something that can:
1. **Introspect the GraphQL schema** to understand available operations
2. **Generate individual tools** for each query and mutation
3. **Handle field selection automatically** to avoid GraphQL validation errors
4. **Provide proper parameter validation** before execution

```mermaid
graph LR
    A[GraphQL API] --> B[Schema Introspection]
    B --> C[Tool Generation]
    C --> D[MCP Server]
    D --> E[AI Agent Tools]
    E --> F[Tool Calls]
    F --> D
    D --> G[GraphQL Queries]
    G --> A
```

### ⚠️ Critical Design Considerations

Before we dive into implementation, there are important caveats to consider:

#### 1. **Mutation Safety**: Should mutations be exposed as tools?
```graphql
# This becomes a tool that could delete data!
type Mutation {
  deleteUser(id: ID!): Boolean
  chargeCard(amount: Float!): PaymentResult  
}
```
**Question**: Should AI agents have unrestricted access to write operations?

#### 2. **Data Shape**: How much data should we return?
```graphql
# Minimal - fast, fits in context
{ id, name }

# Expansive - comprehensive, but large
{ id, name, email, posts { title, content, comments { author, text } } }
```
**Consideration**: Context window limits vs completeness

#### 3. **Query Composition**: Some queries only make sense together
```graphql
# Step 1: Get user ID
user(username: "john") { id }

# Step 2: Get user's posts (needs ID from step 1)  
posts(authorId: "user-123") { title }
```
**Challenge**: Individual tools may not provide complete workflows

### Schema → Tool Translation

Let's see how GraphQL concepts map to AI tool concepts:

| GraphQL Concept | AI Tool Equivalent | Example |
|----------------|-------------------|---------|
| **Query Field** | **Read Tool** | `query_user` → Get user data |
| **Mutation Field** | **Write Tool** | `mutation_createPost` → Create content |
| **Field Arguments** | **Tool Parameters** | `id: ID!` → Required string parameter |
| **Field Description** | **Tool Description** | What the AI agent sees |
| **Return Type** | **Tool Output** | Structured response data |

### Critical Insight: Documentation = Tool Quality

The quality of your GraphQL schema documentation **directly impacts** AI tool effectiveness:

```graphql
# ❌ Poor: AI won't understand what this does
type Query {
  user(id: ID!): User
}

# ✅ Excellent: AI understands purpose and usage  
type Query {
  """
  Fetch a user by their unique identifier. Returns user profile 
  information including name, email, and associated content.
  Used for user lookup and profile management.
  """
  user(
    """The unique identifier for the user to retrieve"""
    id: ID!
  ): User
}
```


In [ ]:
## 🛡️ Chapter 2: Safety First - Queries vs Mutations in AI Context

### The Read/Write Divide

When exposing GraphQL APIs to AI agents, the distinction between queries and mutations becomes **critical for safety**:

#### Queries (Read Operations) ✅ Lower Risk
- **Data retrieval only**
- **Idempotent** - safe to retry
- **No side effects** 
- **Perfect for exploration** - agents can safely experiment

```graphql
type Query {
  # Safe for AI agents
  users: [User]
  posts(limit: Int): [Post]  
  searchContent(query: String!): [SearchResult]
}
```

#### Mutations (Write Operations) ⚠️ Higher Risk  
- **Modify data state**
- **Have side effects** - create, update, delete
- **Potentially expensive** - send emails, charge cards
- **Require governance** - audit trails, approval workflows

```graphql
type Mutation {
  # Requires careful consideration for AI
  createUser(input: CreateUserInput!): User
  deletePost(id: ID!): Boolean
  chargeCard(amount: Float!, cardToken: String!): PaymentResult
}


In [ ]:
### Existing Solutions: The Traditional Approach

Before we dive into our solution, let's examine an existing project that takes a different approach: **[mcp-graphql by blurrah](https://github.com/blurrah/mcp-graphql)** (207 stars, actively maintained).

#### How Traditional MCP GraphQL Works

The traditional approach exposes **only 2 generic tools**:

1. **`introspect-schema`** - Get the schema structure
2. **`query-graphql`** - Execute any GraphQL operation

```javascript
// Traditional approach: One generic tool for everything
{
  name: "query-graphql",
  description: "Execute GraphQL queries against the endpoint",
  inputSchema: {
    type: "object", 
    properties: {
      query: { type: "string" },    // Raw GraphQL string
      variables: { type: "object" }  // Variables object
    }
  }
}
```

#### The Challenge: Expert-Level Knowledge Required

This approach expects the **caller to know**:
- ✅ GraphQL query syntax
- ✅ Exact field names and types
- ✅ Proper field selection for complex types
- ✅ How to handle circular references
- ✅ Parameter requirements and formats

**Example usage:**
```graphql
# AI agent must construct this entire query manually
query GetUser($id: ID!) {
  user(id: $id) {
    id
    name
    email
    posts {
      title
      content
      author {  # Circular reference - how to handle?
        id
        name
      }
    }
  }
}
```

#### Why This Is Limiting for AI Agents

1. **High Cognitive Load**: AI must understand GraphQL syntax deeply
2. **Field Selection Complexity**: Agents struggle with nested object requirements  
3. **Schema Discovery**: Must introspect first, then construct queries
4. **Error-Prone**: Easy to create invalid field selections
5. **No Guidance**: No hints about what each operation does

This is exactly why we need a **different approach** that generates individual, purpose-built tools for each GraphQL operation.


In [ ]:
## 🔬 Chapter 3: Hands-On Examples - Real APIs to AI Tools

### Example A: Real Strapi CMS GraphQL API

Let's start with a real-world Strapi CMS instance to understand the basics.

#### Step 1: Install MCP GraphQL Forge


In [ ]:
# Install the MCP GraphQL Forge package
npm install -g @toolprint/mcp-graphql-forge

# Or run directly with npx (recommended)
npx @toolprint/mcp-graphql-forge --help


In [ ]:
#### Step 2: Real-Life Demo - Let's See It In Action! 

Let's pause here for a **live demonstration** of the GraphQL proxy service in action.

**What we'll observe:**
- Schema introspection process
- Tool generation from GraphQL fields
- Field selection cache building
- Generated tool inspection

#### Step 3: Start with HTTP Transport (Development Mode)

HTTP transport is perfect for development - you can inspect and test tools before integrating with AI agents.


In [ ]:
# 🎬 LIVE DEMO: Watch the GraphQL → MCP transformation happen

# Let's use our real Strapi CMS GraphQL API to see all the concepts in action
GRAPHQL_ENDPOINT="https://patient-song-383e3762d2.strapiapp.com/graphql" \
npx @toolprint/mcp-graphql-forge --transport http --port 3001

# Watch for these key outputs:
# 1. "Introspecting GraphQL schema..." - Schema analysis
# 2. "🗂️ Building field selection cache..." - Solving the field selection problem  
# 3. "📊 Generated field selections for X types" - Circular reference handling
# 4. "Generated Y tools from GraphQL schema" - Individual tools created
# 5. Tool breakdown: queries vs mutations

# Strapi typically generates rich schemas with:
# - Content types (Articles, Users, Categories, etc.)
# - Nested relationships between entities
# - Comprehensive field selections
# - Both queries and mutations for CRUD operations

# This demonstrates:
# ✅ Automatic introspection (caveat #1 addressed)
# ✅ Smart field selection (caveat #2 - balanced data shape)
# ✅ Individual tool generation (caveat #3 - composition challenge)


In [ ]:
### 🔍 Under the Hood: The 5-Step Transformation Process

When you run the command above, here's exactly what happens behind the scenes:

#### **Step 1: Schema Fetching** 📡
```bash
# Output: "Introspecting GraphQL schema..."
```
- **Connects** to your GraphQL endpoint
- **Executes** a standard introspection query
- **Downloads** the complete schema definition including types, fields, and descriptions

#### **Step 2: JSON Schema Generation** 📋
```bash  
# Output: "🗂️ Building field selection cache for all types..."
```
- **Parses** the GraphQL schema into TypeScript objects
- **Converts** GraphQL types to JSON Schema format for MCP
- **Maps** GraphQL scalars (`String`, `Int`, `Boolean`) to JSON types
- **Handles** complex types like enums, inputs, and nested objects

#### **Step 3: Strict Tool Generation** 🛠️
```bash
# Output: "Generated Y tools from GraphQL schema: X query tools, Y mutation tools"
```
- **Creates individual MCP tools** for each GraphQL query and mutation
- **Generates precise input schemas** that match GraphQL parameter requirements exactly
- **Enforces required parameters** (NonNull types) vs optional ones
- **Provides rich descriptions** from GraphQL schema documentation

#### **Step 4: Smart Field Selection** 🎯
```javascript
// Instead of this manual complexity:
query GetArticle($id: ID!) {
  article(id: $id) {
    id
    title
    content
    author {
      id
      name
      // What about circular references?
    }
  }
}

// MCP GraphQL Forge auto-generates optimal selections
```
- **Expands all scalar fields** automatically (`id`, `title`, `content`)
- **Resolves nested objects** with proper field selections
- **Handles circular references** intelligently (User → Posts → Author → User)

#### **Step 5: Fragment Caching System** ⚡
```bash
# Output: "📊 Generated field selections for X types"
# Output: "💾 Field selection cache contains X full selections and Y minimal selections"
```
- **Pre-computes field selections** for every GraphQL type once
- **Caches full fragments** for comprehensive data retrieval
- **Caches minimal fragments** for breaking circular references
- **Ensures consistency** - same type always generates identical queries
- **Optimizes performance** - no re-computation during tool execution

### The Result: Perfect GraphQL Queries, Zero Manual Work

Each generated tool knows exactly:
- ✅ **What parameters to accept** (from GraphQL arguments)
- ✅ **How to validate inputs** (from GraphQL type system)  
- ✅ **Which fields to select** (from cached fragments)
- ✅ **How to handle relationships** (via circular reference detection)

This transforms a complex, error-prone manual process into a **reliable, automated system**.


In [ ]:
### 🕵️ Now Let's Inspect and Test the Generated Tools

With your MCP server running, let's explore the generated tools using the MCP Inspector.

#### Step 1: Start the MCP Inspector


In [ ]:
# In a NEW terminal window (keep the MCP server running in the first)
npx @modelcontextprotocol/inspector

# This will start the inspector on http://localhost:5173
# You'll see output like:
# ➜  Local:   http://localhost:5173/
# ➜  Network: use --host to expose


In [ ]:
#### Step 2: Connect to Your MCP Server

1. **Open your browser** to `http://localhost:5173`
2. **Enter server URL**: `http://localhost:3001/mcp`
3. **Click "Connect"**
4. **Watch the magic**: You'll see all the generated tools appear!

#### Step 3: Explore the Generated Tools

You should see tools like:
- `query_articles` - Get blog posts/articles
- `query_categories` - Get content categories
- `query_users` - Get user information  
- `mutation_createArticle` - Create new content
- And many more based on your Strapi schema

#### Step 4: Test Some Real Endpoints

Let's test a few tools to see the system in action:


In [ ]:
##### 🧪 Test 1: Query Articles (Safe Read Operation)

**Tool**: `query_articles`
**Purpose**: Get all articles from the CMS

**Parameters to try**:
```json
{
  "pagination": {
    "limit": 5
  }
}
```

**What happens behind the scenes**:
```graphql
# MCP GraphQL Forge automatically generates:
query articlesOperation($pagination: PaginationArg) {
  articles(pagination: $pagination) {
    data {
      id
      attributes {
        title
        content
        publishedAt
        # ... all available fields expanded automatically
      }
    }
  }
}
```

##### 🧪 Test 2: Query Single Article by ID

**Tool**: `query_article`  
**Purpose**: Get a specific article

**Parameters to try**:
```json
{
  "id": "1"
}
```

**Note**: See how the tool **knows exactly** which parameters are required vs optional!

##### 🧪 Test 3: Query Categories (Relationship Testing)

**Tool**: `query_categories`
**Purpose**: Get content categories with their related articles

**Parameters to try**:
```json
{
  "pagination": {
    "limit": 3
  }
}
```

**Observe**: How nested relationships are automatically resolved with proper field selections.


In [ ]:
##### ⚠️ Test 4: Mutation Testing (Be Careful!)

**⚠️ WARNING**: Mutations modify your data! Only test if you're comfortable with changes.

**Tool**: `mutation_createArticle`
**Purpose**: Create a new article

**Parameters to try** (if you want to test):
```json
{
  "data": {
    "title": "Test Article from MCP",
    "content": "This article was created through the MCP GraphQL bridge!"
  }
}
```

**Better approach for learning**: 
- Just **inspect the tool parameters** to see the input schema
- **Don't execute** unless you want to create test data
- **Observe** how required fields are marked vs optional ones

### 🎯 What to Observe During Testing

#### Field Selection Magic ✨
- **No manual GraphQL writing** - everything is auto-generated
- **Complete field selections** - no "must specify subfields" errors
- **Consistent structure** - same types always return identical field sets

#### Parameter Validation 🛡️
- **Required parameters** are clearly marked (red asterisks in Inspector)
- **Type validation** happens before GraphQL execution
- **Rich descriptions** help understand what each parameter does

#### Performance Optimization ⚡
- **Fast responses** thanks to field selection caching
- **Consistent queries** - no re-computation of field selections
- **Minimal circular references** - smart handling of complex relationships

#### Error Handling 🚨
- **Clear error messages** when parameters are missing
- **GraphQL validation errors** are caught early
- **Helpful suggestions** for fixing issues

This demonstrates the **power of automatic tool generation** vs manual GraphQL construction!


In [ ]:
# Start the MCP server with your Strapi CMS GraphQL API
# Using our real Strapi instance for demonstration
GRAPHQL_ENDPOINT="https://patient-song-383e3762d2.strapiapp.com/graphql" \
npx @toolprint/mcp-graphql-forge --transport http --port 3001

# You'll see output like:
# 🗂️  Building field selection cache for all types...
# 📊 Generated field selections for X types  
# Generated Y tools from GraphQL schema:r
#   - X query tools (articles, categories, users, etc.)
#   - Y mutation tools (create, update, delete operations)
# GraphQL MCP Server started with HTTP transport on port 3001


In [ ]:
### 🎉 Congratulations! 

You've successfully transformed a Strapi CMS GraphQL API into AI-accessible tools. You've seen:

- ✅ **Automatic schema introspection** and tool generation
- ✅ **Smart field selection** that handles complex relationships  
- ✅ **Parameter validation** that prevents GraphQL errors
- ✅ **Fragment caching** for performance and consistency
- ✅ **Individual tools** instead of generic query interfaces

Now let's explore a more complex example...


In [ ]:
## 🤖 Chapter 3.5: Connecting to AI Clients (Claude/Cursor)

Now that we've tested our GraphQL-to-MCP bridge, let's connect it to actual AI clients where the real magic happens.

### The Transport Switch: HTTP → Stdio

**Important**: AI clients like Claude and Cursor use **stdio transport**, not HTTP.

- **HTTP Transport** = Great for development, testing with Inspector
- **Stdio Transport** = Required for AI client integration

### Step 1: Configure for AI Client Integration


In [ ]:
#### For Claude Desktop

Create or edit your `claude_desktop_config.json` file:

**Location**:
- **macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`
- **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`


In [ ]:
{
  "mcpServers": {
    "strapi-cms": {
      "command": "npx",
      "args": [
        "-y",
        "@toolprint/mcp-graphql-forge"
      ],
      "env": {
        "GRAPHQL_ENDPOINT": "https://patient-song-383e3762d2.strapiapp.com/graphql"
      }
    }
  }
}


In [ ]:
#### For Cursor

Create or edit your `mcp.json` file in your project root:


In [ ]:
### Example B: GitHub GraphQL API (Production Scale)

Now let's try a real-world, complex API. GitHub's GraphQL API is massive and well-documented.

#### Step 1: Get a GitHub Token

You'll need a GitHub Personal Access Token:


In [ ]:
1. Go to GitHub Settings → Developer Settings → Personal Access Tokens
2. Generate a new token with `repo` and `read:org` scopes
3. Copy the token (you'll need it below)

#### Step 2: Start MCP Server with Authentication


In [ ]:
# Replace YOUR_TOKEN with your actual GitHub token
GRAPHQL_ENDPOINT="https://api.github.com/graphql" \
GRAPHQL_AUTH_HEADER="Bearer YOUR_TOKEN" \
npx @toolprint/mcp-graphql-forge --transport http --port 3002

# This will generate 100+ tools from GitHub's massive schema!
# You'll see tools like:
# - query_repository
# - query_user  
# - query_organization
# - mutation_createIssue
# - mutation_addComment
# - And many more...


In [ ]:
## 📝 Chapter 4: Schema Design for AI - Best Practices

### The Documentation Imperative

**Your schema documentation directly determines AI tool quality.** Here are the essential patterns:

#### ✅ Excellent Documentation Pattern

```graphql
type Query {
  """
  Search for repositories based on various criteria. Returns paginated results
  with comprehensive repository information including ownership, statistics,
  and content metadata. Perfect for repository discovery and analysis tasks.
  
  Example: Find popular TypeScript repositories with high star counts.
  """
  searchRepositories(
    """
    Search query using GitHub's search syntax. Supports filters like:
    - language:typescript (filter by programming language)  
    - stars:>1000 (repositories with more than 1000 stars)
    - created:>2023-01-01 (recently created repositories)
    """
    query: String!
    
    """Number of results to return (1-100, default: 10)"""
    limit: Int = 10
    
    """Pagination cursor for getting next page of results"""
    after: String
  ): SearchRepositoryResult!
}
```

#### ❌ Poor Documentation Pattern

```graphql
type Query {
  # AI has no idea what this does or how to use it
  searchRepositories(
    query: String!
    limit: Int
    after: String  
  ): SearchRepositoryResult!
}
```


In [ ]:
### AI-Friendly Documentation Guidelines

#### 1. **Clear Purpose Statement**
Start each field description with what it does and why you'd use it.

```graphql
"""
Create a new user account with email verification. Sends welcome email
and sets up default preferences. Use this for user registration flows.
"""
createUser(input: CreateUserInput!): User
```

#### 2. **Parameter Guidance** 
Explain each parameter's purpose, format, and constraints.

```graphql
"""
User's email address. Must be valid format and not already registered.
Will be used for login and notifications.
"""
email: String!

"""
Password strength: minimum 8 characters, must include numbers and symbols.
Will be securely hashed before storage.
"""  
password: String!
```

#### 3. **Include Examples**
Show realistic usage examples in descriptions.

```graphql
"""
Filter users by role and status. 
Example: role: "admin", active: true returns all active administrators.
"""
filterUsers(role: UserRole, active: Boolean): [User]
```

#### 4. **Error Scenarios**
Document when operations might fail.

```graphql
"""
Delete a user account. Requires admin permissions or self-deletion.
Throws PERMISSION_DENIED if unauthorized.
Throws NOT_FOUND if user doesn't exist.
"""
deleteUser(id: ID!): Boolean
```


In [ ]:
## 🎯 Chapter 5: Toolprint - Smart Tool Orchestration

### The Tool Selection Challenge

When your MCP GraphQL Forge generates 50+ tools from a complex API, **AI agents struggle to choose the right tools**:

- Which tool should I use for "find recent commits"?
- How do I combine multiple tools to complete a task?
- What's the optimal sequence for complex workflows?

### Toolprint Solution: Semantic Tool Discovery

**Toolprint** enhances MCP by adding intelligent tool selection and workflow creation:

#### Semantic Search
Instead of tool names, search by **intent**:

```javascript
// Instead of remembering: "query_repository"
findTool("get repository information")

// Instead of: "mutation_createIssue" 
findTool("report a bug or feature request")

// Instead of: "query_pullRequests"
findTool("review code changes")
```

#### Workflow Creation (Toolprints)
Combine tools into reusable patterns:

```yaml
# Toolprint: "Investigate Repository"
name: investigate_repository
description: "Comprehensive repository analysis"
steps:
  - tool: query_repository
    description: "Get basic repo info"
  - tool: query_commits  
    description: "Check recent activity"
  - tool: query_issues
    description: "Review open issues"
  - tool: query_contributors
    description: "Analyze contributor activity"
```

### Integration with MCP GraphQL Forge

Toolprint automatically enhances your generated tools with semantic metadata, making them more discoverable and easier to orchestrate into powerful workflows.


In [ ]:
## 🚀 Next Steps & Production Readiness

### Production Deployment Checklist

#### 🔐 Security & Authentication
- [ ] Set up proper API authentication (Bearer tokens, API keys)
- [ ] Configure CORS for HTTP transport
- [ ] Enable audit logging for mutation operations
- [ ] Set up rate limiting and quota management

#### 📊 Monitoring & Observability  
- [ ] Track tool usage metrics and performance
- [ ] Monitor GraphQL query complexity and execution time
- [ ] Set up alerts for failed operations
- [ ] Log all mutations for compliance/audit trails

#### 🛡️ Safety Guardrails
- [ ] Start with query-only deployment
- [ ] Implement mutation approval workflows for high-risk operations
- [ ] Set up staging environment for testing tool changes
- [ ] Document acceptable use policies for AI agents

### Key Takeaways

1. **Documentation Quality = Tool Quality**: Invest heavily in GraphQL schema descriptions
2. **Safety First**: Start with queries, carefully govern mutations
3. **Iterative Approach**: Begin simple, add complexity gradually
4. **Monitor Everything**: Track usage, performance, and errors
5. **Toolprint Integration**: Use semantic search for better tool discovery

### Resources

- **MCP GraphQL Forge**: [GitHub Repository](https://github.com/toolprint/mcp-graphql-forge)
- **MCP Documentation**: [Model Context Protocol](https://modelcontextprotocol.io/)
- **Toolprint Platform**: [Smart Tool Orchestration](https://toolprint.ai)
- **GraphQL Best Practices**: [Schema Design Guide](https://graphql.org/learn/best-practices/)

---

**🎉 Congratulations!** You've successfully transformed GraphQL APIs into AI-accessible tools. Your APIs are now ready to be leveraged by AI agents for automation, analysis, and intelligent workflows.
